In [1]:
## Following MATLAB code from http://capture-clarity.org/clarity-based-tractography/

In [1]:
import os

In [2]:
## Parameters (the script loops through all parameters and saves each result automatically)
dogsigmaArr = [1];  # Sigma values for derivative of gaussian filter, recommended value: 0.6 - 1.3 (based on actual data)
gausigmaArr = [2.3];  # Sigma values for gaussian filter, recommended value: 1.3 - 2.3 (based on actual data)
angleArr = [25];  # Angle thresholds for fiber tracking, recommended value: 20 - 30

In [3]:
pwd

u'/Users/Kepler/code/seelviz/Tony/ipynb'

In [4]:
import numpy as np

In [5]:
import math
from scipy import ndimage
import nibabel as nib

In [21]:
## Change later on
# file_path = "/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb/TIFF_stack"
file_path = "/Users/Kepler/code/seelviz/Tony/ipynb/TIFF_stack/page"
directory = os.path.dirname(file_path)

In [22]:
pwd

u'/Users/Kepler/code/seelviz/Tony/ipynb/TIFF_stack'

In [24]:
# cd TIFF_stack/

SyntaxError: invalid syntax (<ipython-input-24-c80a05be5086>, line 2)

In [25]:
cd page/

/Users/Kepler/code/seelviz/Tony/ipynb/TIFF_stack/page


In [26]:
from PIL import Image
from numpy import matlib
from scipy import signal

# Loading the TIFF stack

In [29]:
im = Image.open('page1.tiff')
# im.show()

imarray = np.array(im)
print(imarray.shape)
# print(imarray)

im = Image.open('page2.tiff')
# im.show()

imarray2 = np.array(im)
print(imarray2.shape)

(600, 800, 4)
(600, 800, 4)


In [39]:
s = np.stack((imarray, imarray2), axis=3)
print s.shape

s2 = np.stack((s, imarray2), axis=3)
print s2.shape

c = np.concatenate((imarray[...,np.newaxis], imarray2[...,np.newaxis]),axis=3)
print c.shape

(600, 800, 4, 2)


ValueError: all input arrays must have the same shape

In [16]:
def tiff_to_array(input_path):
    im = Image.open(input_path)
    # im.show()

    imarray = np.array(im)
    # print(imarray)
    
    return imarray

In [ ]:
for filename in os.listdir(file_path):
    if filename.endswith(".tiff"):
        # print(os.path.join(directory, filename))
        im_list = []
        im_arr = tiff_to_array(file_name)
        im_list.append(im_arr)
        
    s = np.stack(im_list, axis=3)
    print s.shape

In [ ]:
def tiff_stack_to_array(input_path):
    """input_path should lead to a directory"""
    for filename in os.listdir(file_path):
    if filename.endswith(".tiff"):
        # print(os.path.join(directory, filename))
        im_list = []
        im_arr = tiff_to_array(file_name)
        im_list.append(im_arr)
        
    s = np.stack(im_list, axis=3)
    print s.shape
    return s

In [10]:
'''
Function to generate derivatives of Gaussian kernels, in either 1D, 2D, or 3D.
Source code in MATLAB obtained from Qiyuan Tian, Stanford University, September 2015
Edited to work in Python by Tony.
'''

def doggen(sigma):
    halfsize = np.ceil(3 * np.max(sigma))
    x = range(np.single(-halfsize), np.single(halfsize + 1));  # Python colon is not inclusive at end, while MATLAB is.
    dim = len(sigma);
    
    if dim == 1:
        X = np.array(x);  # Remember that, by default, numpy arrays are elementwise multiplicative
        k = -X * np.exp(-X**2/(2 * sigma**2));
        
    elif dim == 2:
        [X, Y] = np.meshgrid(x, x);
        k = -X * np.exp(-X**2/(2*sigma[0]^2) * np.exp(-Y**2))
        
    elif dim == 3:
        [X, Y, Z] = np.meshgrid(x, x, x);
        X = X.transpose(0, 2, 1);  # Obtained through vigorous testing (see below...)
        Y = Y.transpose(2, 0, 1);
        Z = Z.transpose(2, 1, 0);
        
        X = X.astype(float);
        Y = Y.astype(float);
        Z = Z.astype(float);
        k = -X * np.exp(np.divide(-np.power(X, 2), 2 * np.power(sigma[0], 2))) * np.exp(np.divide(-np.power(Y,2), 2 * np.power(sigma[1],2))) * np.exp(np.divide(-np.power(Z,2), 2 * np.power(sigma[2],2)))
        
    else:
        print 'Only supports up to 3 dimensions'
        
    return np.divide(k, np.sum(np.abs(k[:])));

In [11]:
'''
Function to generate Gaussian kernels, in 1D, 2D and 3D.
Source code in MATLAB obtained from Qiyuan Tian, Stanford University, September 2015
Edited to work in Python by Tony. 
'''

def gaussgen(sigma):
    halfsize = np.ceil(3 * max(sigma));
    x = range(np.single(-halfsize), np.single(halfsize + 1));

    dim = len(sigma);

    if dim == 1:
        k = np.exp(-x**2 / (2 * sigma^2));
    
    elif dim == 2:
        [X, Y] = np.meshgrid(x, x);
        k = np.exp(-X**2 / (2 * sigma[0]**2)) * np.exp(-Y**2 / (2 * sigma[1]**2)); 
    
    elif dim == 3:
        [X, Y, Z] = np.meshgrid(x, x, x);
        X = X.transpose(0, 2, 1);  # Obtained through vigorous testing (see below...)
        Y = Y.transpose(2, 0, 1);
        Z = Z.transpose(2, 1, 0);
        
        X = X.astype(float);  # WHY PYTHON?
        Y = Y.astype(float);
        Z = Z.astype(float);
        k = np.exp(-X**2 / (2 * sigma[0]**2)) * np.exp(-Y**2 / (2 * sigma[1]**2)) * np.exp(-Z**2 / (2 * sigma[2]**2));
    
    else:
        print 'Only supports up to dimension 3'

    return np.divide(k, np.sum(np.abs(k)));

In [12]:
# Set up results directory
if not os.path.exists(directory):
    os.makedirs(directory)

# im = Image.open('page1.tiff')  # Needs to be changed to dynamically go down list of fnDataArr (currently just loads same test image)
# Omitted: channel data (red/green - our CLARITY data was single channel, so no channel data loaded.)
ones = np.single(100 * np.ones((10, 10, 10)));  #data is hard coded to be np.ones

# image has a line of intensity value 100 at col = 4, depth = 4
line = np.zeros((10, 10, 10))
line[:,2,2] = 100
line = np.single(line)

In [13]:
types_of_data = [ones, line];
tensorFSLTEST = [];

for x in types_of_data:
    img_data = x;
    #print img_data[:, :, 3];
    
    for jj in range(len(dogsigmaArr)):
        dogsigma = dogsigmaArr[jj];
        print "Start DoG Sigma on " + str(dogsigma);

        # Generate dog kernels
        dogkercc = doggen([dogsigma, dogsigma, dogsigma]);
        dogkercc = np.transpose(dogkercc, (0, 2, 1));  # annoying

        #print dogkercc.shape;
        #print dogkercc[:, :, 0];

        dogkerrr = np.transpose(dogkercc, (1, 0, 2));

        #print dogkerrr[:, :, 0];
        dogkerzz = np.transpose(dogkercc, (0, 2, 1));

        #print dogkerzz[:, :, 0];

        # Compute gradients
        grr = signal.convolve(img_data, dogkerrr, 'same');

        #print grr[:, :, 0];

        gcc = signal.convolve(img_data, dogkercc, 'same');

        #print gcc[:, :, 0];

        gzz = signal.convolve(img_data, dogkerzz, 'same');

        #print gzz[:, :, 0];

        # Compute gradient products
        gprrrr = np.multiply(grr, grr);

        #print gprrrr[:, :, 0];

        gprrcc = np.multiply(grr, gcc);

        #print gprrcc[:, :, 0];

        gprrzz = np.multiply(grr, gzz);

        #print gprrzz[:, :, 0]

        gpcccc = np.multiply(gcc, gcc);
        gpcczz = np.multiply(gcc, gzz);
        gpzzzz = np.multiply(gzz, gzz);

        # Compute gradient amplitudes
        # print ga.dtype;
        ga = np.sqrt(gprrrr + gpcccc + gpzzzz);

        #print ga[:, :, 0];

        print "GA SHAPE:"
        print ga.shape;

        # Convert numpy ndarray object to Nifti data type
        gradient_amplitudes_data = nib.Nifti1Image(ga, affine=np.eye(4));

        # Save gradient amplitudes image 
        nib.save(gradient_amplitudes_data, 'gradient_amplitudes.nii');

        # Compute gradient vectors
        gv = np.concatenate((grr[..., np.newaxis], gcc[..., np.newaxis], gzz[..., np.newaxis]), axis = 3);
        
        #print gv[:, :, 0, 0];
        
        gv = np.divide(gv, np.tile(ga[..., None], [1, 1, 1, 3]));
        #print gv[:, :, 0, 1];

        print "GV SHAPE:"
        print gv.shape;

        # Convert numpy ndarray object to Nifti data type
        gradient_vectors_data = nib.Nifti1Image(gv, affine=np.eye(4));

        # Save gradient vectors
        nib.save(gradient_vectors_data, 'gradient_vectors.nii');

        # Compute structure tensor
        for kk in range(len(gausigmaArr)):
            gausigma = gausigmaArr[kk];
            print "Start Gauss Sigma with gausigma = " + str(gausigma);

            print "Generating Gaussian kernel..."
            gaussker = np.single(gaussgen([gausigma, gausigma, gausigma]));

            #print gaussker[:, :, 0];

            print "Blurring gradient products..."
            gprrrrgauss = signal.convolve(gprrrr, gaussker, "same");
            
            #print gprrrrgauss[:, :, 0];
            
            gprrccgauss = signal.convolve(gprrcc, gaussker, "same");
            
            #print gprrccgauss[:, :, 0];
            
            gprrzzgauss = signal.convolve(gprrzz, gaussker, "same");
            gpccccgauss = signal.convolve(gpcccc, gaussker, "same");
            gpcczzgauss = signal.convolve(gpcczz, gaussker, "same");
            gpzzzzgauss = signal.convolve(gpzzzz, gaussker, "same");

            print "Saving a copy for this Gaussian sigma..."
            tensorfsl = np.concatenate((gprrrrgauss[..., np.newaxis], gprrccgauss[..., np.newaxis], gprrzzgauss[..., np.newaxis], gpccccgauss[..., np.newaxis], gpcczzgauss[..., np.newaxis], gpzzzzgauss[..., np.newaxis]), axis = 3);
            tensorFSLTEST.append(tensorfsl);
            
             # Convert numpy ndarray object to Nifti data type
            tensor_fsl_data = nib.Nifti1Image(tensorfsl, affine=np.eye(4));

            nib.save(tensor_fsl_data, "dogsigma_" + str(jj) + "gausigma_" + str(kk) + 'tensorfsl.nii');

    print 'Complete!'

Start DoG Sigma on 1
GA SHAPE:
(10, 10, 10)
GV SHAPE:
(10, 10, 10, 3)
Start Gauss Sigma with gausigma = 2.3
Generating Gaussian kernel...
Blurring gradient products...
Saving a copy for this Gaussian sigma...
Complete!
Start DoG Sigma on 1
GA SHAPE:
(10, 10, 10)
GV SHAPE:
(10, 10, 10, 3)
Start Gauss Sigma with gausigma = 2.3
Generating Gaussian kernel...
Blurring gradient products...


/Users/Kepler/anaconda/envs/ml/lib/python2.7/site-packages/ipykernel/__main__.py:76: RuntimeWarning: invalid value encountered in divide


Saving a copy for this Gaussian sigma...
Complete!


In [14]:
## ONES:
tensorfsl = tensorFSLTEST[0];

In [15]:
## Compare tensor_fsl_data with imported values from MATLAB
print tensorfsl.shape  # numpy

(10, 10, 10, 6)


In [16]:
MATLAB_output = nib.load("/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb/test_MATLAB_10by10by10_outputs/dogsig1_gausig2.3/test_MATLAB_tensorfsl_dogsig1_gausig2.3.nii")

In [17]:
print MATLAB_output.shape

(10, 10, 10, 6)


In [18]:
MATLAB_np_array = MATLAB_output.get_data()
print MATLAB_np_array.shape

(10, 10, 10, 6)


In [19]:
print tensorfsl[:, :, :, 0];

[[[ 109.85943033  147.37287499  179.26563687  201.37589584  212.3919397
    212.3919397   201.37589584  179.26563687  147.37287499  109.85943033]
  [ 147.37287499  197.69594847  240.47905765  270.13925545  284.91692228
    284.91692228  270.13925545  240.47905765  197.69594847  147.37287499]
  [ 179.26563687  240.47905765  292.52080083  328.5997214   346.57540288
    346.57540288  328.5997214   292.52080083  240.47905765  179.26563687]
  [ 201.37589584  270.13925545  328.5997214   369.12854264  389.32130808
    389.32130808  369.12854264  328.5997214   270.13925545  201.37589584]
  [ 212.3919397   284.91692228  346.57540288  389.32130808  410.61869621
    410.61869621  389.32130808  346.57540288  284.91692228  212.3919397 ]
  [ 212.3919397   284.91692228  346.57540288  389.32130808  410.61869621
    410.61869621  389.32130808  346.57540288  284.91692228  212.3919397 ]
  [ 201.37589584  270.13925545  328.5997214   369.12854264  389.32130808
    389.32130808  369.12854264  328.5997214   

In [20]:
print MATLAB_np_array[:, :, :, 0];

[[[ 109.85939789  147.37295532  179.26557922  201.37585449  212.39183044
    212.39196777  201.37599182  179.26565552  147.37287903  109.85943604]
  [ 147.37289429  197.69589233  240.47901917  270.13919067  284.91690063
    284.91680908  270.13919067  240.47918701  197.69593811  147.37290955]
  [ 179.26560974  240.47901917  292.52096558  328.59942627  346.57531738
    346.57531738  328.59970093  292.52087402  240.47904968  179.26560974]
  [ 201.37588501  270.13912964  328.59945679  369.12838745  389.32110596
    389.32110596  369.12866211  328.59973145  270.13916016  201.37582397]
  [ 212.39186096  284.91671753  346.57528687  389.3210144   410.61871338
    410.61868286  389.32144165  346.57556152  284.91680908  212.391922  ]
  [ 212.39189148  284.91677856  346.57519531  389.32107544  410.6187439
    410.61865234  389.32144165  346.57556152  284.9168396   212.39190674]
  [ 201.37590027  270.13916016  328.59954834  369.12850952  389.3213501
    389.32125854  369.12860107  328.59979248  2

In [21]:
truth_boolean = np.isclose(tensorfsl, MATLAB_np_array, rtol = 1e-4)
print truth_boolean;

[[[[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  ..., 
  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ Tr

In [22]:
correct_number = np.sum(truth_boolean == True);  # Total possible = 6000

In [23]:
# Percent Correct (at 1e-4 level) for uniform data:
print correct_number / 6000.0;

1.0


In [24]:
## LINE:
tensorfsl = tensorFSLTEST[1];
MATLAB_output = nib.load("/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb/test_MATLAB_line/dogsig1_norm_gausig2.3/test_MATLAB_norm_tensorfsl_dogsig1_gausig2.3.nii")

In [25]:
MATLAB_np_array = MATLAB_output.get_data()
print MATLAB_np_array.shape

(10, 10, 10, 6)


In [26]:
print tensorfsl[:, :, :, 0];

[[[  5.26199848e-01   6.81899421e-01   7.43445967e-01   6.81924586e-01
     5.26244011e-01   3.41676162e-01   1.86653157e-01   8.57955680e-02
     3.31638995e-02   1.04143515e-02]
  [  6.81899421e-01   8.83669618e-01   9.63427448e-01   8.83702228e-01
     6.81956652e-01   4.42776220e-01   2.41882777e-01   1.11181993e-01
     4.29769106e-02   1.34958996e-02]
  [  7.43445967e-01   9.63427448e-01   1.05038403e+00   9.63463002e-01
     7.43508363e-01   4.82740098e-01   2.63714516e-01   1.21217004e-01
     4.68558996e-02   1.47140060e-02]
  [  6.81924586e-01   8.83702228e-01   9.63463002e-01   8.83734840e-01
     6.81981819e-01   4.42792560e-01   2.41891703e-01   1.11186096e-01
     4.29784966e-02   1.34963976e-02]
  [  5.26244011e-01   6.81956652e-01   7.43508363e-01   6.81981819e-01
     5.26288177e-01   3.41704839e-01   1.86668822e-01   8.58027687e-02
     3.31666829e-02   1.04152256e-02]
  [  3.41676162e-01   4.42776220e-01   4.82740098e-01   4.42792560e-01
     3.41704839e-01   2.21859

In [27]:
print MATLAB_np_array[:, :, :, 0];

[[[  5.26199698e-01   6.81899428e-01   7.43445933e-01   6.81924462e-01
     5.26243806e-01   3.41676176e-01   1.86653152e-01   8.57955441e-02
     3.31639014e-02   1.04143489e-02]
  [  6.81899488e-01   8.83669734e-01   9.63427186e-01   8.83702159e-01
     6.81956470e-01   4.42776203e-01   2.41882831e-01   1.11181967e-01
     4.29769158e-02   1.34958960e-02]
  [  7.43445933e-01   9.63426888e-01   1.05038357e+00   9.63462889e-01
     7.43508041e-01   4.82739955e-01   2.63714552e-01   1.21216975e-01
     4.68559042e-02   1.47140063e-02]
  [  6.81924582e-01   8.83702099e-01   9.63462710e-01   8.83734524e-01
     6.81981742e-01   4.42792594e-01   2.41891727e-01   1.11186072e-01
     4.29784879e-02   1.34963933e-02]
  [  5.26243806e-01   6.81956649e-01   7.43508160e-01   6.81981623e-01
     5.26287913e-01   3.41704875e-01   1.86668798e-01   8.58027712e-02
     3.31666842e-02   1.04152225e-02]
  [  3.41676295e-01   4.42776114e-01   4.82740015e-01   4.42792535e-01
     3.41704756e-01   2.21859

In [28]:
truth_boolean = np.isclose(tensorfsl, MATLAB_np_array, rtol=1e-4)
print truth_boolean;

[[[[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True False  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True False  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True False  True]
   [ True  True False  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  ..., 
  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ Tr

In [29]:
correct_number = np.sum(truth_boolean == True);  # Total possible = 6000

In [30]:
# Percent Correct (at 1e-4 level):
print correct_number / 6000.0;

0.975666666667
